# Проект для «Викишоп»

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Случайный лес</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#*BERT" data-toc-modified-id="*BERT-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>*BERT</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузитье и подготовить данные.
2. Обучить разные модели. 
3. Сделатье выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
# загрузим все необходимые библиотеки
import time
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Сохраним датасет в переменную **commemts**.

In [2]:
comments = pd.read_csv('C:\\Users\\503so\\OneDrive\\Desktop\\praktikum-to-git\\13_toxic_comments.csv')

Посмотрим на первые 5 строк таблицы.

In [3]:
comments.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


И изучим общую информацию о имеющемся датасете.

In [4]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [5]:
comments['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Наш датасет содержит 159571 строку (объект) и 2 столбца: обучающий признак - текст комментария, и целевой признак - принадлежность комментария к категории "токсичных".

Также видим, что наблюдается ощутимый дисбаланс классов: 143346 объектов относятся к категории *не токсичных* (класс 0), и всего 16225 - к категории *токсичных* (класс 1). Или 89.8% и 10.2% соответственно.

Создадим список `corpus`, в котором сохраним все комментарии.

In [6]:
corpus = list(comments['text'])

Напишем функции **lemm** и **clear_text**. Первая будет возвращать леммы слов в комментариях, вторая - очищать текст от "посторонних" символов.

In [7]:
def lemm(text):
    lemm_list = nltk.stem.WordNetLemmatizer()
    return "".join(lemm_list.lemmatize(word) for word in text)
        
    
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text) 
    return " ".join(text.split())

Создадим новый корпус `corpus_clear` на основе исходного корпуса с "очищенными" предложениями.

In [8]:
%%time
corpus_clear = []
for text in corpus:
    corpus_clear.append(clear_text(lemm(text)))

CPU times: user 5min 34s, sys: 794 ms, total: 5min 34s
Wall time: 5min 39s


In [2]:
list(map(lambda x: x.lower(), ["asD", "Asd"]))

['asd', 'asd']

Посмотрим на первые два предложения исходного корпуса и корпуса очищенных текстов.

In [10]:
corpus[:2]

["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
 "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"]

In [11]:
corpus_clear[:2]

['Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They weren t vandalisms just closure on some GAs after I voted at New York Dolls FAC And please don t remove the template from the talk page since I m retired now',
 'D aww He matches this background colour I m seemingly stuck with Thanks talk January UTC']

Загрузим из библиотеки **nltk** стоп-слова для языка "*english*".

In [12]:
stop_words = set(stopwords.words('english'))


В качестве обучающего набора данных выберем результат TF-IDF-векторизации.

In [13]:
features = corpus_clear

In [14]:
target = comments['toxic']

При разделении выборок на обучающую и тестовую, установим размер тестовой выборки, равный 0.3.

In [15]:
features_train, features_test = train_test_split(features, test_size=0.3, shuffle=False)
target_train, target_test = train_test_split(target, test_size=0.3, shuffle=False)

Создадим переменную `count_tf_idf` для выполнения TF-IDF-векторизации корпуса текстов.

In [16]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words)

Обучим векторизатор на обучающей выборке "очищенного" корпуса текстов.

In [17]:
%%time
tf_idf_train = count_tf_idf.fit_transform(features_train)
tf_idf_test = count_tf_idf.transform(features_test)

CPU times: user 11.1 s, sys: 116 ms, total: 11.2 s
Wall time: 11.2 s


## Обучение

Напишем функцию `toxic_score`, которая будет вычислять и выводить на экран три метрики точности предсказаний: метрику f1_score, полноту и точность. Поскольку задача модели - идентифицировать "токсичные" комментарии, чтобы отправить их на модерацию, в связке показателей точность-полнота важнее представляется именно полнота. Лучше, если максимальное количество "токсичных" комментариев будет отправлено на модерацию. При этом, если пострадает точность, то на модерацию будут отправлены комментарии, ложно тдентифицированные моделью как токсичные. В данном случае можно говорить о конфликте скорости и качества: при большем количестве сообщений, отправленных на модерацию, на их обработку уйдет больше времени, но у пользователей сайта будет меньше шансов наткнуться на неприемлемый контент.

In [18]:
def toxic_score(test, predictions):
    f1 = f1_score(test, predictions)
    recall = recall_score(test, predictions)
    precision = precision_score(test, predictions)
    print('Значение метрики f1_score: {:.3f}'.format(f1))
    print('Значение полноты: {:.3f}'.format(recall))
    print('Значение точности: {:.3f}'.format(precision))

### Логистическая регрессия

Поскольку мы имеем дело с ярко выраженным дисбалансом классов, передадим модели логистической регрессии параметр `class_weight='balanced'`.

In [19]:
model = LogisticRegression(class_weight='balanced')

Выведем на экран размерности полученных выборок.

In [20]:
print(tf_idf_train.shape)
print(tf_idf_test.shape)
print(target_train.shape)
print(target_test.shape)

(111699, 137131)
(47872, 137131)
(111699,)
(47872,)


In [21]:
%%time
model.fit(tf_idf_train, target_train)
predictions = model.predict(tf_idf_test)
toxic_score(target_test, predictions)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Значение метрики f1_score: 0.755
Значение полноты: 0.845
Значение точности: 0.682
CPU times: user 8.33 s, sys: 5.03 s, total: 13.4 s
Wall time: 13.4 s


In [22]:
print('Количество токсичных комментариев по предсказаниям:', pd.Series(predictions).value_counts()[1])
print('Реальное количество токсичных комментариев:', pd.Series(target_test).value_counts()[1])

Количество токсичных комментариев по предсказаниям: 6006
Реальное количество токсичных комментариев: 4844


На предсказаниях логистической регрессии мы получили величину метрики **f1_score**, равную **0.755**.
Также видим, что модель отнесла к "токсичным" большее количество комментариев, чем есть на самом деле (при том, что какие-то могла пропустить как приемлемые). Однако это меньшая из двух зол.

### Дерево решений

In [23]:
%%time
best_result = 0
best_depth = 0
for depth in range(1,10):
    model_t = DecisionTreeClassifier(random_state=503350, max_depth=depth)
    model_t.fit(tf_idf_train, target_train)
    predictions_t = model_t.predict(tf_idf_test)
    score_t = f1_score(target_test, predictions_t)
    if score_t > best_result:
        best_result = score_t
        best_depth = depth
        
print('Лучший результат:', best_result)
print('Лучшая глубина дерева:', best_depth)

Лучший результат: 0.5731570061902082
Лучшая глубина дерева: 9
CPU times: user 37.6 s, sys: 26.2 ms, total: 37.7 s
Wall time: 37.8 s


In [24]:
model_tree = DecisionTreeClassifier(random_state=503350, max_depth = best_depth)

In [25]:
%%time
model_tree.fit(tf_idf_train, target_train)
predictions_tree = model_tree.predict(tf_idf_test)
toxic_score(target_test, predictions_tree)

Значение метрики f1_score: 0.573
Значение полноты: 0.421
Значение точности: 0.900
CPU times: user 7.38 s, sys: 0 ns, total: 7.38 s
Wall time: 7.42 s


### Случайный лес

In [27]:
%%time
best_result = 0
best_depth = 0
best_est = 0
for depth_f in range(5, 15):
    for n_est in range(10, 100, 10):
        model_f = RandomForestClassifier(class_weight='balanced', 
                                         random_state=503350,
                                         max_depth=depth_f,
                                         n_estimators=n_est)
        model_f.fit(tf_idf_train, target_train)
        predictions_f = model_f.predict(tf_idf_test)
        score_f = f1_score(target_test, predictions_f)
        if score_f > best_result:
            best_result_f = score_f
            best_depth_f = depth_f
            best_est = n_est
        
print('Лучший результат:', best_result_f)
print('Лучшая глубина дерева:', best_depth_f)
print('Лучшее количество наблюдателей:', best_est)

Лучший результат: 0.36912812190064015
Лучшая глубина дерева: 14
Лучшее количество наблюдателей: 90
CPU times: user 5min 28s, sys: 261 ms, total: 5min 28s
Wall time: 5min 33s


In [28]:
model_forest = RandomForestClassifier(class_weight='balanced', 
                                      random_state=503350,
                                      max_depth = best_depth_f,
                                      n_estimators = best_est)

In [29]:
%%time
model_forest.fit(tf_idf_train, target_train)
predictions_forest = model_forest.predict(tf_idf_test)
toxic_score(target_test, predictions_forest)

Значение метрики f1_score: 0.369
Значение полноты: 0.845
Значение точности: 0.236
CPU times: user 8.39 s, sys: 0 ns, total: 8.39 s
Wall time: 8.5 s


## Выводы

По результатам выполненной работы можем заключить:
1. Логистическая регрессия поепзала результат, лучший, чем дерево решений: 0.755 против 0.573. Результат случайного леса оказался еще хуже - 0.37 (возможно, модель переобучилась).
2. Не всегда имеет смысл делать лемматизацию текстов корпуса: лемматизация не только занимает большое количество времени, но и может снизить качество модели в частном случае. Т.о. стоит начинать с самой простой модели, и постепенно добавалять шаги предобработки и гиперпараметры модели, следя за результатом метрики.